In [1]:
import pandas as pd
dictionary=pd.read_csv("Dictionary.csv")
dictionary.head()

,Word,Sequence Number,Word Count,Word Proportion,Average Proportion,Std Dev,Doc Count,Negative,Positive,Uncertainty,Litigious,Constraining,Superfluous,Interesting,Modal,Irr_Verb,Harvard_IV,Syllables,Source
0,AARDVARK,1,81,5.690000e-09,3.070000e-09,5.780000e-07,45,0,0,0,0,0,0,0,0,0,0,2,12of12inf
1,AARDVARKS,2,2,1.400000e-10,8.220000e-12,7.840000e-09,1,0,0,0,0,0,0,0,0,0,0,2,12of12inf
2,ABACI,3,8,5.620000e-10,1.690000e-10,7.100000e-08,7,0,0,0,0,0,0,0,0,0,0,3,12of12inf
3,ABACK,4,5,3.510000e-10,1.730000e-10,7.530000e-08,5,0,0,0,0,0,0,0,0,0,0,2,12of12inf
4,ABACUS,5,1752,1.230000e-07,1.200000e-07,1.110000e-05,465,0,0,0,0,0,0,0,0,0,0,3,12of12inf


In [2]:
Negative=list(dictionary[dictionary.Negative>0]["Word"])
Positive=list(dictionary[dictionary.Positive>0]["Word"])
Uncertainty=list(dictionary[dictionary.Uncertainty>0]["Word"])
Litigious=list(dictionary[dictionary.Litigious>0]["Word"])
Constraining=list(dictionary[dictionary.Constraining>0]["Word"])
Superfluous=list(dictionary[dictionary.Superfluous>0]["Word"])
Interesting=list(dictionary[dictionary.Interesting>0]["Word"])
Modal1=list(dictionary[dictionary.Modal==1]["Word"])
Modal2=list(dictionary[dictionary.Modal==2]["Word"])
Modal3=list(dictionary[dictionary.Modal==3]["Word"])


In [3]:
from elasticsearch import Elasticsearch
es = Elasticsearch(["http://search-banktweets-oe3qkimmg7aavxn2djiwrm5bi4.us-east-1.es.amazonaws.com:80"])
res=es.search(index="banktweets", doc_type="tweets",scroll="1m",body={"from":0, "size":100, "query": {"match": {"text":"Wells Fargo"}}})
scrollId=res["_scroll_id"]

In [4]:
import re
dataset=[]
scrollres=es.scroll(scroll_id=scrollId,scroll="1m") 
scrollres

{'_scroll_id': 'DnF1ZXJ5VGhlbkZldGNoBQAAAAAACw0ZFjNjbGhIOU1QUldDSVpqTm1BTVhfUncAAAAAAAsNGhYzY2xoSDlNUFJXQ0laak5tQU1YX1J3AAAAAAALDRsWM2NsaEg5TVBSV0NJWmpObUFNWF9SdwAAAAAACw0cFjNjbGhIOU1QUldDSVpqTm1BTVhfUncAAAAAAAsNHRYzY2xoSDlNUFJXQ0laak5tQU1YX1J3',
 '_shards': {'failed': 0, 'successful': 5, 'total': 5},
 'hits': {'hits': [{'_id': 'AVvPjuqbQJg5hlQOP5cj',
    '_index': 'banktweets',
    '_score': 3.7020044,
    '_source': {'text': 'who has wells fargo??',
     'time': 1493835901000,
     'user': 'March 23rd ♈'},
    '_type': 'tweets'},
   {'_id': 'AVvXolB7QJg5hlQOQACC',
    '_index': 'banktweets',
    '_score': 3.7020044,
    '_source': {'text': 'Fuck Wells Fargo',
     'time': 1493971390000,
     'user': 'paperchace 💸'},
    '_type': 'tweets'},
   {'_id': 'AVvXpLkPQJg5hlQOQACn',
    '_index': 'banktweets',
    '_score': 3.7020044,
    '_source': {'text': '@Nav_Yeet_Sosa Wow Wells Fargo',
     'time': 1493971548000,
     'user': 'augustine'},
    '_type': 'tweets'},
   {'_id': 'AVvYy2MyQJg

In [5]:
while( len(scrollres["hits"]["hits"])>0):    
    for hit in scrollres['hits']['hits']:
        neg,pos,unc,lit,con,sup,intr,mod1,mod2,mod3=0,0,0,0,0,0,0,0,0,0
        p = re.compile("[^a-zA-Z]?")
        stringArray=p.split(hit["_source"]["text"])
        for word in stringArray:
            WORD=word.upper()
            if WORD in Negative:
                neg=neg+1
            if WORD in Positive:
                pos=pos+1
            if WORD in Uncertainty:
                unc=unc+1
            if WORD in Litigious:
                lit=lit+1
            if WORD in Constraining:
                con=con+1
            if WORD in Superfluous:
                sup=sup+1
            if WORD in Interesting:
                intr=intr+1
            if WORD in Modal1:
                mod1=mod1+1
            if WORD in Modal2:
                mod2=mod2+1
            if WORD in Modal3:
                mod3=mod3+1
        array=[hit["_source"]["time"],neg,pos,unc,lit,con,sup,intr,mod1,mod2,mod3]
        dataset.append(array)
    scrollres=es.scroll(scroll_id=scrollId,scroll="1m")Wells Fargo


E:\Study\Python\Anaconda03\lib\site-packages\ipykernel\__main__.py:5: FutureWarning: split() requires a non-empty pattern match.


In [6]:
df=pd.DataFrame(dataset,columns=['timestamp','neg', 'pos', 'unc', 'lit', 'con','sup','intr','Mod1','Mod2','Mod3'])
df.head()

,timestamp,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3
0,1493835901000,0,0,0,0,0,0,0,0,0,0
1,1493971390000,0,0,0,0,0,0,0,0,0,0
2,1493971548000,0,0,0,0,0,0,0,0,0,0
3,1493990859000,0,0,0,0,0,0,0,0,0,0
4,1494008077000,0,0,0,0,0,0,0,0,0,0


In [8]:
from datetime import datetime
def timetransfer(ts):
    return datetime.fromtimestamp(ts/1000+600).strftime('%Y-%m-%d %H:%M:%S')
df["time"]=df["timestamp"].map(lambda t:timetransfer(t))
df.head()


,timestamp,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3,time
0,1493799479000,0,0,0,0,0,0,0,0,0,0,2017-05-03 04:27:59
1,1492627326000,0,0,0,0,0,0,0,0,0,0,2017-04-19 14:52:06
2,1492646742000,0,0,0,0,0,0,0,0,0,0,2017-04-19 20:15:42
3,1492700678000,0,0,0,0,0,0,0,0,0,0,2017-04-20 11:14:38
4,1492722269000,0,0,0,0,0,0,0,0,0,0,2017-04-20 17:14:29


In [9]:
df=df.set_index(pd.DatetimeIndex(df['time'])).sort_index().drop("timestamp",axis=1)
df["count"]=1
df=df.resample("5T").sum()
df.head()


,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3,count
time,,,,,,,,,,,
2017-04-16 18:25:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0
2017-04-16 18:30:00,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2017-04-16 18:35:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0
2017-04-16 18:40:00,2.0,5.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,12.0
2017-04-16 18:45:00,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6.0


In [10]:
import pandas as pd
import numpy as np
import urllib
import datetime as dt
import matplotlib.pyplot as plt
 
def get_google_data(symbol, period, window):
    url_root = 'http://www.google.com/finance/getprices?i='
    url_root += str(period) + '&p=' + str(window)
    url_root += 'd&f=d,o,h,l,c,v&df=cpct&q=' + symbol
    response = urllib.request.urlopen(url_root)
    data = str(response.read()).split('\\n')
#actual data starts at index = 7
#first line contains full timestamp,
#every other line is offset of period from timestamp
    parsed_data = []
    anchor_stamp = ''
    end = len(data)
    for i in range(7, end):
        cdata = data[i].split(',')
        if 'a' in cdata[0]:
#first one record anchor timestamp
            anchor_stamp = cdata[0].replace('a', '')
            cts = int(anchor_stamp)
        else:
            try:
                coffset = int(cdata[0])
                cts = int(anchor_stamp) + (coffset * period)
                parsed_data.append((dt.datetime.fromtimestamp(float(cts)), float(cdata[1]), float(cdata[2]), float(cdata[3]), float(cdata[4]), float(cdata[5])))
            except:
                pass # for time zone offsets thrown into data
    df = pd.DataFrame(parsed_data)
    df.columns = ['ts', 'o', 'h', 'l', 'c', 'v']
    df.index = df.ts
    del df['ts']
    return df

In [11]:
BOA = get_google_data('BAC', 300, 14)
BOA.head()

,o,h,l,c,v
ts,,,,,
2017-04-18 09:35:00,22.8975,23.15,22.885,23.130,7520432.0
2017-04-18 09:40:00,22.9300,22.99,22.870,22.895,5130022.0
2017-04-18 09:45:00,22.9750,22.99,22.930,22.935,4203378.0
2017-04-18 09:50:00,22.8950,23.01,22.890,22.975,4462407.0
2017-04-18 09:55:00,22.7900,22.89,22.785,22.890,3014132.0


In [12]:
BOA["C-O"]=BOA["c"]-BOA["o"]
def binary(v):
    if(v>0):
        return 1
    else:
        return 0
BOA["Target"]=BOA["C-O"].map(lambda t:binary(t))  
BOA.head()

,o,h,l,c,v,C-O,Target
ts,,,,,,,
2017-04-18 09:35:00,22.8975,23.15,22.885,23.130,7520432.0,0.2325,1
2017-04-18 09:40:00,22.9300,22.99,22.870,22.895,5130022.0,-0.0350,0
2017-04-18 09:45:00,22.9750,22.99,22.930,22.935,4203378.0,-0.0400,0
2017-04-18 09:50:00,22.8950,23.01,22.890,22.975,4462407.0,0.0800,1
2017-04-18 09:55:00,22.7900,22.89,22.785,22.890,3014132.0,0.1000,1


In [13]:
BOAData=BOA.join(df,how="left").dropna().drop(["o","h","l","c","v","C-O"],axis=1)
BOAData

,Target,neg,pos,unc,lit,con,sup,intr,Mod1,Mod2,Mod3,count
ts,,,,,,,,,,,,
2017-04-18 09:35:00,1,12.0,1.0,0.0,4.0,0.0,0.0,0.0,0.0,1.0,0.0,44.0
2017-04-18 09:40:00,0,6.0,8.0,3.0,2.0,0.0,0.0,0.0,4.0,0.0,3.0,42.0
2017-04-18 09:45:00,0,5.0,2.0,0.0,2.0,0.0,0.0,1.0,0.0,0.0,0.0,40.0
2017-04-18 09:50:00,1,3.0,2.0,2.0,0.0,0.0,0.0,1.0,1.0,1.0,2.0,46.0
2017-04-18 09:55:00,1,6.0,3.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,31.0
2017-04-18 10:00:00,0,3.0,3.0,2.0,0.0,0.0,0.0,0.0,0.0,2.0,2.0,34.0
2017-04-18 10:05:00,1,2.0,1.0,1.0,0.0,0.0,0.0,2.0,0.0,1.0,0.0,25.0
2017-04-18 10:10:00,1,11.0,2.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,38.0
2017-04-18 10:15:00,1,5.0,5.0,0.0,1.0,0.0,0.0,0.0,4.0,1.0,0.0,28.0


AUC plotting function

In [23]:
import matplotlib
import matplotlib.pyplot as plt
from sklearn.metrics import roc_curve, auc
%matplotlib inline

def plotUnivariateROC(preds, truth, label_string):
    #1. call the roc_curve function to get the ROC X and Y values
    fpr, tpr, thresholds = roc_curve(truth, preds)
    #2. Input fpr and tpr into the auc function to get the AUC
    roc_auc = auc(fpr, tpr)
    #we are doing this as a special case because we are sending unfitted predictions into the function
    if roc_auc < 0.5:
        fpr, tpr, thresholds = roc_curve(truth, -1 * preds)
        roc_auc = auc(fpr, tpr)
        print('Warning:{} is worse than random!'.format(label_string))
    #chooses a random color for plotting
    c = (np.random.rand(), np.random.rand(), np.random.rand())
    #create a plot and set some options
    plt.plot(fpr, tpr, color = c, label = label_string + ' (AUC = %0.3f)' % roc_auc)
    

    plt.plot([0, 1], [0, 1], 'k--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.0])
    plt.xlabel('FPR')
    plt.ylabel('TPR')
    plt.title('ROC')
    plt.legend(loc="lower right")
    
    return roc_auc


Train and fit the data to Model

In [14]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
from sklearn.metrics import confusion_matrix, roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.grid_search import GridSearchCV
from sklearn.cross_validation import KFold

BOAData_shuf=BOAData.reindex(np.random.permutation(BOAData.index))
kfolds = KFold(BOAData_shuf.shape[0], n_folds = 5)
X = BOAData_shuf.drop('Target', 1)
Y =  BOAData_shuf['Target']

#1st, set up a paramater grid
param_grid_lr = {'min_samples_split':[20*i for i in range(1, 10)], 'min_samples_leaf':[i*2 for i in range(1, 10)]}

#2nd, call the GridSearchCV class, use LogisticRegression and 'log_loss' for scoring
lr_grid_search = GridSearchCV(DecisionTreeClassifier(), param_grid_lr, cv = kfolds, scoring = 'roc_auc') 
lr_grid_search.fit(X, Y)

best_1 = lr_grid_search.best_score_
print(best_1)

E:\Study\Python\Anaconda03\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
E:\Study\Python\Anaconda03\lib\site-packages\sklearn\grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


0.5417323118544151


In [37]:
param_grid_lr = {'C':[10**i for i in range(-10, 10)], 'penalty':['l1','l2']}

#2nd, call the GridSearchCV class, use LogisticRegression and 'log_loss' for scoring
lr_grid_search = GridSearchCV(LogisticRegression(), param_grid_lr, cv = kfolds, scoring = 'roc_auc') 
lr_grid_search.fit(X, Y)

best_1 = lr_grid_search.best_score_
print(best_1)

0.5145479321760843


In [ ]:
param_grid_lr = {'C':[10**i for i in range(-10, 10)], 'kernel':['linear']}

#2nd, call the GridSearchCV class, use LogisticRegression and 'log_loss' for scoring
lr_grid_search = GridSearchCV(SVC(), param_grid_lr, cv = kfolds, scoring = 'roc_auc') 
lr_grid_search.fit(X, Y)

best_1 = lr_grid_search.best_score_
print(best_1)